This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [2]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = ''

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [3]:
# First, import the relevant modules
import requests

In [4]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned

r = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2016-01-01&end_date=2016-01-01&api_key='+API_KEY)

In [5]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure

print(r.json()) # gives us an idea of what we're working with

# Top level: dataset_data
# Second level: limit, transform, column_index, column_names, start_date, end_date, frequency, daily, data, collapse
    # column_names has Date, Open, High, Low, Close, Change, Traded Volume, Turnover, 
        # Last Price of the Day, Daily Traded Units, Daily Turnover
    # data has values corresponding to the column names in double [[]]

{'dataset_data': {'limit': None, 'transform': None, 'column_index': None, 'column_names': ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'], 'start_date': '2016-01-01', 'end_date': '2016-01-01', 'frequency': 'daily', 'data': [['2016-01-01', None, 28.73, 28.275, 28.55, 0.15, 31307.0, 893928.0, None, None, None]], 'collapse': None, 'order': None}}


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [6]:
# Step 1: Collect data for entire year of 2017.
r = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-01-01&end_date=2017-12-31&api_key='+API_KEY)

In [7]:
# Step 2: Convert the json object into a Python Dictionary
r_python = dict(r.json()) # convert json object to a dictionary

In [8]:
# Step 3. Calculate the highest and lowest opening prices in this period
colnames = r_python['dataset_data']['column_names'] # extract the column names for the data values
    # we see that 'Open' is the second value in data, so the first index
    # also, to build a dictionary, we should probably use dates as keys (first value, 0 index)

data = r_python['dataset_data']['data'] # extract the data values
open_data = dict() # create an empty dictionary to add to

for index, value in enumerate(data): # enumerate the data and loop through it
    if value[1] is not None: # there are "None" fields in the data, so we only want those that are not none
        open_data[value[0]] = value[1] # save the value to the dictionary with the date as the key
            # and the open value as the value

print('The minimum opening value in a day is: ' + str(min(open_data.values()))) # print the minimum
print('The maximum opening value in a day is: ' + str(max(open_data.values()))) # print the maximum

The minimum opening value in a day is: 34.0
The maximum opening value in a day is: 53.11


In [9]:
# Step 4: What was the largest change in a day (based on High and Low prices)

# print(colnames) 
    # We see that high and Low refers to the third and fourth values in data

change_data = dict() # create a new dictionary to add to

for index, value in enumerate(data): # enumerate the data and loop through it
    change_data[value[0]] = (value[2]-value[3]) # save the value to the dictionary with the date as the key
        # and the value as the difference in the high - low prices

print('The minimum change in one day is: ' + str(min(change_data.values()))) # print the minimum
print('The maximum change in one day is: ' + str(max(change_data.values()))) # print the maximum

The minimum change in one day is: 0.18999999999999773
The maximum change in one day is: 2.8100000000000023


In [10]:
# Step 5: What is the largest change between two days, based on closing price?

# print(colnames)
    # We see that closing price refers to the fifth element (4th index) in the data
    
two_days = dict() # create a new dictionary to add to

for index, value in enumerate(data): # enumerate the data and loop through it
    if index > 0 : # if the index is greater than 0 (the first value), we can calculate the
        # change in closing price from one day to the previous day
        change = data[index-1][4] - value[4]  # the change in closing price can be is today's price - yesterday's price
            # note that the data are from most recent to most historic, so we need to subtract the previous index's value from
            # the current index's value.
        two_days[str(value[0])+ ' to '+ str(data[index-1][0])] = abs(change)
            # we want to add the value to the dictionary with the date strings as the keys 'yesterday to today'
            # and save the values as the absolute change between those closing prices

print('The largest change between any two days is: ' + str(max(two_days.values()))) # print out the maximum difference

The largest change between any two days is: 2.559999999999995


In [11]:
# Step 6: What is average daily trading volume during the year?

# print(colnames)
    # We see that the trading volume refers to the 7th element (6th index) in the data

trading_volume = dict() # create a new dictionary to add to

for index, value in enumerate(data): # enumerate the data and loop through it
    trading_volume[value[0]] = value[6] # add the values to the dictionary with the date as the key
        # and the volume as the value
 
print('The average daily trading volume during 2017 is: ' + str(sum(trading_volume.values())/len(trading_volume))) 
    # print the average for all trading 

The average daily trading volume during 2017 is: 89124.33725490196


In [12]:
# Step 7: What was the median trading volume during the year?

def median_calc(values): 
    """Calculates the median."""
    index_value = (len(values)/2)-1 # index starts at 0
    if len(value) % 2 != 0: 
        return (values[int(index_value-0.5)] + values[int(index_value+0.5)])/2
    else: 
        return values[index_value]

trading_volume_values = sorted(trading_volume.values()) # Sort the values in order from smallest to largest

print('The median daily trading volume during 2017 is: ' + str(median_calc(trading_volume_values)))

The median daily trading volume during 2017 is: 75630.5
